In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
finals = pd.read_csv('semi_and_final_features.csv')
finals.columns = ['Account_ID', 'EVENT_ID', 'TRANSACTION_COUNT', 'STATUS_ID',
       'INPLAY_BET', 'AVG_BET_SIZE', 'MAX_BET_SIZE', 'MIN_BET_SIZE',
       'STDEV_BET_SIZE']

In [3]:
submit = pd.read_csv('sample_submission_bet_size.csv')

In [4]:
submit.columns = ['Account_ID', 'Prediction']

In [6]:
import glob

In [7]:
filenames = glob.glob('Datathon*.csv')

In [8]:
pre_dat = pd.DataFrame({})
for ind, filename in enumerate(filenames):
    print(ind)
    tmp_dat = pd.read_csv(filename)
    pre_dat = pre_dat.append(tmp_dat)    
pre_dat.columns = ['BET_ID', 'BET_TRANS_ID', 'MATCH_BET_ID', 'ACCOUNT_ID',
       'COUNTRY_OF_RESIDENCE_NAME', 'PARENT_EVENT_ID', 'EVENT_ID', 'MATCH',
       'EVENT_NAME', 'EVENT_DT', 'OFF_DT', 'BID_TYP', 'STATUS_ID',
       'PLACED_DATE', 'TAKEN_DATE', 'SETTLED_DATE', 'CANCELLED_DATE',
       'SELECTION_NAME', 'PERSISTENCE_TYPE', 'BET_PRICE', 'PRICE_TAKEN',
       'INPLAY_BET', 'BET_SIZE', 'PROFIT_LOSS']
peek_dat = pd.read_csv('DSMDatathon_samplefile.txt', sep='\t')
pre_dat = peek_dat.append(pre_dat)

0
1

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



2
3
4


In [10]:
pre_dat.loc[:, 'BID_TYP'] = pre_dat.BID_TYP.str.strip(' ')
pre_dat.loc[:, 'STATUS_ID'] = pre_dat.STATUS_ID.str.strip(' ')
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'correct_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'correct_profit_loss'] = - pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'correct_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'correct_profit_loss'] = - pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [11]:
dat = pre_dat.loc[~pre_dat.EVENT_ID.isin([101183237, 101183757, 101183885, 101184013])]

In [14]:
label_dat = pre_dat.loc[pre_dat.EVENT_ID.isin([101183237, 101183757, 101183885, 101184013])]

In [15]:
label_dat = label_dat.loc[(label_dat.STATUS_ID == 'S').values]

In [16]:
labels = label_dat.groupby('ACCOUNT_ID')['correct_profit_loss'].sum()

In [19]:
labels = labels.apply(lambda x: int(x > 0))

In [47]:
# frequency model
dat.loc[:, 'correct_profit_loss'] = dat.correct_profit_loss.fillna(0)

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [102]:
dat = dat.loc[(dat.STATUS_ID == 'S').values]

In [268]:
# creat same features as in the new data
dat_by_account_bet = dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])

In [269]:
tran_freq = dat_by_account_bet['BET_ID'].size()

In [270]:
bet_size = dat_by_account_bet['BET_SIZE'].sum()

In [271]:
tran_freq.name = 'tran_freq'
bet_size.name = 'bet_size'

In [272]:
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()

In [273]:
tran_freq.head()

,ACCOUNT_ID,EVENT_ID,tran_freq
0,1011,101092958,1
1,1534,101092958,2
2,1887,101092958,1
3,2155,101092958,2
4,2166,101092958,3


In [274]:
bet_size.head()

,ACCOUNT_ID,EVENT_ID,bet_size
0,1011,101092958,49.4950
1,1534,101092958,6.9293
2,1887,101092958,397.9398
3,2155,101092958,49.4950
4,2166,101092958,178.1820


In [275]:
features = pd.merge(tran_freq, bet_size, on=['ACCOUNT_ID', 'EVENT_ID'])

In [276]:
features.head()

,ACCOUNT_ID,EVENT_ID,tran_freq,bet_size
0,1011,101092958,1,49.4950
1,1534,101092958,2,6.9293
2,1887,101092958,1,397.9398
3,2155,101092958,2,49.4950
4,2166,101092958,3,178.1820


In [201]:
labels.shape

(9003,)

In [277]:
features_by_account = features.groupby('ACCOUNT_ID')

In [278]:
tran_freq = features_by_account['tran_freq'].mean()
bet_size = features_by_account['bet_size'].mean()

In [279]:
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()

In [280]:
features = pd.merge(tran_freq, bet_size, on=['ACCOUNT_ID'])

In [281]:
features.head()

,ACCOUNT_ID,tran_freq,bet_size
0,1011,1,49.4950
1,1534,2,6.9293
2,1887,1,397.9398
3,2155,2,49.4950
4,2166,3,178.1820


In [282]:
features = features.join(labels, on='ACCOUNT_ID', how='right')

In [283]:
features = features.rename(columns={'correct_profit_loss': 'label'})

In [284]:
features.head()

,ACCOUNT_ID,tran_freq,bet_size,label
225,1000002,13.500000,1868.673437,0
227,1000004,2.300000,218.497670,0
228,1000005,24.166667,7134.352333,0
229,1000006,6.062500,107.404848,1
232,1000009,7.000000,6479.176066,0


In [285]:
features = features.fillna(0)

In [230]:
features.loc[:, 'bet_size'] = features.bet_size / features.tran_freq

In [76]:
from sklearn import cross_validation

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [286]:
models = {
    'lg': LogisticRegression(),
    'rf': RandomForestClassifier(n_estimators=20),
    'bdt': AdaBoostClassifier()}

In [287]:
scores = []
for model_name, model in models.items():
    print(model_name)
    clf = model
    score = cross_validation.cross_val_score(clf, features[['tran_freq', 'bet_size']], 
                                             features.label, cv=5, scoring='f1')
    scores.append(score)

rf
lg
bdt


In [288]:
scores

[array([ 0.43100996,  0.41260315,  0.42276423,  0.42402315,  0.25646552]),
 array([ 0.07653061,  0.06020942,  0.03698811,  0.03195739,  0.06451613]),
 array([ 0.39292035,  0.47281713,  0.40909091,  0.36779506,  0.49382716])]

In [237]:
new_dat = pd.read_csv('semi_and_final_features.csv')
new_dat.columns = ['Account_ID', 'EVENT_ID', 'TRANSACTION_COUNT', 'STATUS_ID',
       'INPLAY_BET', 'AVG_BET_SIZE', 'MAX_BET_SIZE', 'MIN_BET_SIZE',
       'STDEV_BET_SIZE']

In [314]:
new_dat.head()

,EVENT_ID,TRANSACTION_COUNT,STATUS_ID,INPLAY_BET,AVG_BET_SIZE,MAX_BET_SIZE,MIN_BET_SIZE,STDEV_BET_SIZE,total
Account_ID,,,,,,,,,
1004240,101187238,11,S,Y,272.349536,638.772899,6.387729,263.609320,2995.844895
1004240,101187238,3,S,Y,638.772899,638.772899,638.772899,0.000000,1916.318696
1004240,101187943,1,S,Y,31.767044,31.767044,31.767044,0.000000,31.767044
1004240,101187943,3,S,Y,56.384385,73.852024,31.767044,21.934563,169.153155
1004240,101191295,12,S,Y,601.317013,1938.380991,9.614370,673.428584,7215.804154


In [311]:
new_dat = new_dat.set_index('Account_ID')

In [313]:
new_dat = new_dat.loc[submit.Account_ID]

In [315]:
new_dat = new_dat.reset_index()

In [316]:
new_dat.loc[:, 'total'] = new_dat.TRANSACTION_COUNT * new_dat.AVG_BET_SIZE

In [317]:
new_dat = new_dat.loc[(new_dat.STATUS_ID == 'S').values]

In [318]:
new_dat_by_account = new_dat.groupby(['Account_ID', 'EVENT_ID'])

In [319]:
tran_freq = new_dat_by_account['TRANSACTION_COUNT'].sum()

In [320]:
bet_size = new_dat_by_account['total'].sum()

In [321]:
tran_freq.name = 'tran_freq'
bet_size.name = 'bet_size'

In [322]:
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()

In [323]:
features2 = pd.merge(tran_freq, bet_size, on=['Account_ID', 'EVENT_ID'])

In [324]:
features2.head()

,Account_ID,EVENT_ID,tran_freq,bet_size
0,1000002,101187238,45,6675.475635
1,1000002,101187943,69,12563.978200
2,1000002,101191295,13,986.819016
3,1000004,101187943,1,943.100000
4,1000005,101187238,10,426.802500


In [325]:
features_by_account = features2.groupby('Account_ID')

In [326]:
tran_freq = features_by_account['tran_freq'].mean()
bet_size = features_by_account['bet_size'].mean()

In [327]:
tran_freq = tran_freq.reset_index()
bet_size = bet_size.reset_index()

In [328]:
features2 = pd.merge(tran_freq, bet_size, on=['Account_ID'])

In [329]:
rfc = RandomForestClassifier(n_estimators=30)

In [330]:
rfc.fit(features[['tran_freq', 'bet_size']], features['label'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [348]:
pred = rfc.predict_proba(features2[['tran_freq', 'bet_size']])

In [341]:
pred

array([[ 0.26666667,  0.73333333],
       [ 0.76666667,  0.23333333],
       [ 0.53333333,  0.46666667],
       ..., 
       [ 0.16666667,  0.83333333],
       [ 0.66666667,  0.33333333],
       [ 0.73333333,  0.26666667]])

In [344]:
pred = rfc.predict(features2[['tran_freq', 'bet_size']])

In [354]:
pred2 = pd.DataFrame({'Account_ID': submit.Account_ID, 'Prediction': pred[:, 0]})

In [352]:
pred2.head()

,Account_ID,Prediction
0,1004240,0.733333
1,1009742,0.233333
2,1003908,0.466667
3,1020888,0.500000
4,1010074,0.166667


In [355]:
pred2.to_csv('easy_ml_model.csv', index=False)